In [ ]:
'''!pip uninstall -y transformers peft bitsandbytes
!pip install -q transformers==4.30.0 accelerate datasets evaluate scikit-learn'''

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 128.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

import pandas as pd

Mounted at /content/drive


In [ ]:
#final_v2 = pd.read_csv("/content/drive/MyDrive/SEM 2/NLP/GP/final_v2.csv")
final_v2 = pd.read_csv("data/final_v2.csv")
test_raw = final_v2[final_v2['rate_difference'].isna()]

In [3]:
test_df = test_raw[['date','clean_text','country']]
test_df.head()

,date,clean_text,country
4773,2025-01-06 00:00:00,"January 06, 2025 An Assessment of the Economy ...",United States
4774,2025-01-08 00:00:00,"January 08, 2025 Challenges Facing Central Ban...",United States
4775,2025-01-09 00:00:00,"January 09, 2025 Reflections on 2024: Monetary...",United States
4778,2025-01-31 00:00:00,"January 31, 2025 Brief Remarks on the Economy,...",United States
4779,2025-02-04 00:00:00,"February 04, 2025 U.S. Economic Outlook and Mo...",United States


In [ ]:
import pandas as pd
import torch
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from tqdm import tqdm


# Model paths
#two_class_model_path = "/content/drive/MyDrive/SEM 2/NLP/GP/03_MAT/roberta_2_class_best_macro_f1"
two_class_model_path = "data/roberta_2_class_best_macro_f1"
#five_class_model_path = "/content/drive/MyDrive/SEM 2/NLP/GP/03_MAT/roberta_5_class_best_macro_f1_weighted"
five_class_model_path = "data/roberta_5_class_best_macro_f1_weighted"

# Load tokenizers
tokenizer_2_class = RobertaTokenizerFast.from_pretrained(two_class_model_path)
tokenizer_5_class = RobertaTokenizerFast.from_pretrained(five_class_model_path)

# Load models
model_2_class = RobertaForSequenceClassification.from_pretrained(two_class_model_path)
model_5_class = RobertaForSequenceClassification.from_pretrained(five_class_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2_class.to(device)
model_5_class.to(device)

# Prepare test data
texts = test_df['clean_text'].tolist()

# Label mappings
two_class_label_mapping = {0: 'cut', 1: 'rise'}
five_class_label_mapping = {
    0: 'Immediate Cut',
    1: 'Immediate Rise',
    2: 'Upcoming Cut',
    3: 'Upcoming Rise',
    4: 'Stable'
}

# Predict with 2-class model
print("\n Predicting with 2-class model...")
two_class_preds = []
model_2_class.eval()
with torch.no_grad():
    for text in tqdm(texts, desc="2-Class Inference"):
        inputs = tokenizer_2_class(text, truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)
        outputs = model_2_class(**inputs)
        pred = torch.argmax(outputs.logits, axis=1).item()
        two_class_preds.append(two_class_label_mapping[pred])

# Predict with 5-class model
print("\n Predicting with 5-class model...")
five_class_preds = []
model_5_class.eval()
with torch.no_grad():
    for text in tqdm(texts, desc="5-Class Inference"):
        inputs = tokenizer_5_class(text, truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)
        outputs = model_5_class(**inputs)
        pred = torch.argmax(outputs.logits, axis=1).item()
        five_class_preds.append(five_class_label_mapping[pred])

# Combine results
test_df['2_class_prediction'] = two_class_preds
test_df['5_class_prediction'] = five_class_preds

test_df.head()


 Predicting with 2-class model...


2-Class Inference: 100%|██████████| 38/38 [00:01<00:00, 31.15it/s]



 Predicting with 5-class model...


5-Class Inference: 100%|██████████| 38/38 [00:00<00:00, 42.81it/s]
<ipython-input-4-7aa6a168e6e0>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['2_class_prediction'] = two_class_preds
<ipython-input-4-7aa6a168e6e0>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['5_class_prediction'] = five_class_preds


,date,clean_text,country,2_class_prediction,5_class_prediction
4773,2025-01-06 00:00:00,"January 06, 2025 An Assessment of the Economy ...",United States,rise,Immediate Rise
4774,2025-01-08 00:00:00,"January 08, 2025 Challenges Facing Central Ban...",United States,rise,Immediate Cut
4775,2025-01-09 00:00:00,"January 09, 2025 Reflections on 2024: Monetary...",United States,rise,Immediate Rise
4778,2025-01-31 00:00:00,"January 31, 2025 Brief Remarks on the Economy,...",United States,rise,Immediate Rise
4779,2025-02-04 00:00:00,"February 04, 2025 U.S. Economic Outlook and Mo...",United States,rise,Immediate Rise


In [ ]:
# prompt: fai un csv basato su  test_df e chiamalo temporal_aware_classification

#test_df.to_csv('/content/drive/MyDrive/SEM 2/NLP/GP/03_MAT/temporal_aware_classification.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score

# Load the data
#train_df1 = pd.read_csv('/content/drive/MyDrive/SEM 2/NLP/GP/03_MAT/train_df1.csv')
train_df1 = pd.read_csv('data/train_df1.csv')

# Extract features and labels
X_all = train_df1['clean_text'].values
y_all = train_df1['label'].values

# Encode labels if needed
if not np.issubdtype(y_all.dtype, np.number):
    label_mapping = {'cut': 0, 'rise': 1}
    y_all = np.array([label_mapping[label] for label in y_all])

# ------------------------- #
#  Train on first 14/15, test on last 1/15
# ------------------------- #
n = len(X_all)
cutoff = int(n * 14 / 15)

X_train = X_all[:cutoff]
y_train = y_all[:cutoff]

X_test = X_all[cutoff:]
y_test = y_all[cutoff:]

# ------------------------- #
#  Pipeline
# ------------------------- #
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 6))),
    ('chi2', SelectKBest(score_func=chi2, k=30000)),
    ('clf', LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000, class_weight='balanced'))
])

# Train on 14/15
pipeline.fit(X_train, y_train)

# Predict on 1/15
y_pred = pipeline.predict(X_test)


print(" Classification Report on 1/15 holdout set:")
print(classification_report(y_test, y_pred, target_names=['cut', 'rise']))
print(f" Macro F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")

 Classification Report on 1/15 holdout set:
              precision    recall  f1-score   support

         cut       1.00      0.02      0.04       272
        rise       0.15      1.00      0.26        47

    accuracy                           0.16       319
   macro avg       0.57      0.51      0.15       319
weighted avg       0.87      0.16      0.07       319

 Macro F1 Score: 0.1482


In [9]:
import pandas as pd

# Extract the texts
X_test = test_df['clean_text'].values

# Predict the labels using the existing trained pipeline
predictions = pipeline.predict(X_test)

# Convert numerical labels back to original
inverse_label_mapping = {0: 'cut', 1: 'rise'}
predicted_labels = [inverse_label_mapping[label] for label in predictions]

# Create the results dataframe
results_df = test_df.copy()
results_df['predicted_label'] = predicted_labels

# Display the results
from IPython.display import display
display(results_df.head())

,date,clean_text,country,2_class_prediction,5_class_prediction,predicted_label
4773,2025-01-06 00:00:00,"January 06, 2025 An Assessment of the Economy ...",United States,rise,Immediate Rise,rise
4774,2025-01-08 00:00:00,"January 08, 2025 Challenges Facing Central Ban...",United States,rise,Immediate Cut,rise
4775,2025-01-09 00:00:00,"January 09, 2025 Reflections on 2024: Monetary...",United States,rise,Immediate Rise,rise
4778,2025-01-31 00:00:00,"January 31, 2025 Brief Remarks on the Economy,...",United States,rise,Immediate Rise,rise
4779,2025-02-04 00:00:00,"February 04, 2025 U.S. Economic Outlook and Mo...",United States,rise,Immediate Rise,rise


In [11]:
results_df = results_df.drop(columns=['2_class_prediction', '5_class_prediction'])

In [ ]:
# prompt: salva a csv results_df chiamandolo temporal_aware classification_HULK

#results_df.to_csv('/content/drive/MyDrive/SEM 2/NLP/GP/03_MAT/temporal_aware_classification_HULK.csv', index=False)

In [14]:
results_df

,date,clean_text,country,predicted_label
4773,2025-01-06 00:00:00,"January 06, 2025 An Assessment of the Economy ...",United States,rise
4774,2025-01-08 00:00:00,"January 08, 2025 Challenges Facing Central Ban...",United States,rise
4775,2025-01-09 00:00:00,"January 09, 2025 Reflections on 2024: Monetary...",United States,rise
4778,2025-01-31 00:00:00,"January 31, 2025 Brief Remarks on the Economy,...",United States,rise
4779,2025-02-04 00:00:00,"February 04, 2025 U.S. Economic Outlook and Mo...",United States,rise
4781,2025-02-05 00:00:00,"February 05, 2025 Bank Regulation in 2025 and ...",United States,rise
4782,2025-02-05 00:00:00,"February 05, 2025 Do Non-inflationary Economic...",United States,rise
4783,2025-02-05 00:00:00,"February 05, 2025 Bank Regulation in 2025 and ...",United States,rise
4784,2025-02-05 00:00:00,"February 05, 2025 Bank Regulation in 2025 and ...",United States,rise
4785,2025-02-07 00:00:00,"February 07, 2025 Entrepreneurship and Aggrega...",United States,rise
